<a href="https://colab.research.google.com/github/devluckhenry/assistente_de_escrita/blob/main/Projeto_IA_Alura_%2B_Google_Gemini_Assistente_de_Escrita_para_Historias_Consistentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Instala Requisitos
!pip install -q google-genai   # Instala a biblioteca para usar os modelos Gemini.
!pip install -q google-adk     # Instala o framework de agentes do Google.

# Configura a API Key do Google Gemini
import os
from google.colab import userdata

# Tenta obter a chave da API do Gemini armazenada nas configurações do Colab ('Secrets').
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except KeyError:
    print(" A chave da API do Google Gemini não foi encontrada nas configurações do Colab.")
    print(" Por favor, siga as instruções para adicioná-la em 'Secrets'.")
    # Se a chave não for encontrada, o código pode não funcionar corretamente.

from google.adk.agents import Agent          # Classe para criar agentes (nossas 'mentes' especializadas).
from google.adk.runners import Runner        # Classe para executar os agentes.
from google.adk.sessions import InMemorySessionService # Para gerenciar sessões de conversa com os agentes.
from google.genai import types               # Para definir tipos de conteúdo (como texto das mensagens).
import textwrap                              # Para formatar texto (como adicionar indentação).
from IPython.display import display, Markdown, FileLink # Para exibir conteúdo formatado e links para arquivos no Colab.
from google.colab import files               # Para interagir com arquivos no ambiente do Colab (upload e download).

import warnings
warnings.filterwarnings("ignore") # Ignora avisos para uma saída mais limpa.

# Função auxiliar para formatar texto em Markdown para melhor visualização no Colab.
def to_markdown(text):
    text = text.replace('•', '  *') # Adapta marcadores de lista.
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)) # Indenta o texto para parecer uma citação.

In [15]:
# Funções para lidar com o arquivo do livro

# Função para carregar o rascunho do livro. Agora permite carregar um arquivo do usuário.
def carregar_rascunho():
    uploaded = files.upload() # Abre uma janela para o usuário selecionar um arquivo do seu computador.
    nome_arquivo = next(iter(uploaded)) if uploaded else None # Obtém o nome do arquivo carregado.
    if nome_arquivo:
        try:
            with open(nome_arquivo, "r", encoding="utf-8") as f: # Abre o arquivo para leitura com codificação UTF-8.
                return f.read(), nome_arquivo # Retorna o conteúdo do arquivo e seu nome.
        except Exception as e:
            print(f"Erro ao ler o arquivo: {e}")
            return "", None # Em caso de erro, retorna vazio e None.
    else:
        print("Nenhum arquivo de rascunho foi carregado. Iniciando com um novo livro.")
        return "", "rascunho_livro.txt" # Se nenhum arquivo for carregado, inicia com um novo livro.

# Função para salvar o rascunho atual do livro em um arquivo.
def salvar_rascunho(texto, nome_arquivo="rascunho_livro.txt"):
    with open(nome_arquivo, "w", encoding="utf-8") as f: # Abre o arquivo para escrita (ou cria se não existir).
        f.write(texto) # Escreve o texto fornecido no arquivo.

In [16]:
# Função principal para gerar o próximo capítulo com o Gemini

def escrever_proximo_capitulo(rascunho, instrucoes_usuario):
    # Cria um agente com o modelo Gemini Pro e instruções detalhadas.
    agente_escritor = Agent(
        name="agente_escritor_livro",
        model="gemini-2.0-flash", # Usamos um modelo mais capaz para geração de texto longo.
        instruction=f"""
        Você é um assistente de escrita de livros.
        Leia o rascunho fornecido e, seguindo as instruções do usuário, escreva o próximo capítulo.
        Mantenha a coerência com o estilo de escrita, o fluxo da história e os eventos anteriores.

        Rascunho atual:
        ```
        {rascunho}
        ```

        Instruções para o próximo capítulo:
        ```
        {instrucoes_usuario}
        ```

        Escreva APENAS o próximo capítulo, formatado como "Capítulo X\n[Texto do capítulo]".
        """,
        description="Agente para escrever o próximo capítulo de um livro."
    )

    # A entrada para o agente são as instruções do usuário e o rascunho atual.
    entrada = f"Instruções: {instrucoes_usuario}\nRascunho:\n{rascunho}"

    # Cria um serviço de sessão em memória para o agente.
    session_service = InMemorySessionService()
    # Cria uma nova sessão para a interação com o agente.
    session = session_service.create_session(app_name=agente_escritor.name, user_id="user_livro", session_id="sessao_livro")
    # Cria um 'Runner' para executar o agente.
    runner = Runner(agent=agente_escritor, app_name=agente_escritor.name, session_service=session_service)
    # Formata a mensagem do usuário para enviar ao agente.
    content = types.Content(role="user", parts=[types.Part(text=entrada)])
    # Variável para armazenar a resposta final do agente.
    resposta_final = ""
    # Executa o agente e itera sobre os eventos da resposta.
    for event in runner.run(user_id="user_livro", session_id="sessao_livro", new_message=content):
        # Verifica se o evento é a resposta final do agente.
        if event.is_final_response():
            # Itera sobre as partes do conteúdo da resposta.
            for part in event.content.parts:
                # Se a parte for texto, adiciona à resposta final.
                if part.text:
                    resposta_final += part.text + "\n"
    # Retorna a resposta final, removendo espaços em branco extras no início e fim.
    return resposta_final.strip()

In [ ]:
# Loop principal da aplicação

print("📚 Iniciando o Assistente de Escrita de Livros 📚")

# Permite ao usuário carregar um arquivo de rascunho.
rascunho_atual, nome_arquivo_rascunho = carregar_rascunho()

# Se um rascunho foi carregado, exibe uma mensagem.
if rascunho_atual:
    print(f"\n📖 Rascunho '{nome_arquivo_rascunho}' carregado.\n")
    display(to_markdown(rascunho_atual))
else:
    print("\n📄 Iniciando um novo livro.\n")

# Loop principal para interação com o usuário.
while True:
    # Solicita ao usuário as instruções para o próximo capítulo.
    instrucoes_proximo_capitulo = input("\n✏️ Descreva como você quer o próximo capítulo (ou digite 'finalizar_livro'): ")
    # Se o usuário quer finalizar, sai do loop.
    if instrucoes_proximo_capitulo.lower() == "finalizar_livro":
        print("\n🎉 Finalizando a escrita do livro.")
        # Salva o rascunho final.
        salvar_rascunho(rascunho_atual, nome_arquivo_rascunho)
        # Oferece o arquivo para download.
        files.download(nome_arquivo_rascunho)
        print(f"\n📥 O arquivo '{nome_arquivo_rascunho}' foi baixado.")
        break
    # Se o usuário forneceu instruções, gera o próximo capítulo.
    elif instrucoes_proximo_capitulo:
        print("\n⏳ Gerando o próximo capítulo...")
        # Chama a função para escrever o próximo capítulo usando o Gemini.
        proximo_capitulo = escrever_proximo_capitulo(rascunho_atual, instrucoes_proximo_capitulo)
        print("\n✨ Resultado do Gemini:\n")
        # Exibe o resultado formatado.
        display(to_markdown(proximo_capitulo))

        # Pergunta ao usuário se deseja adicionar o capítulo ao livro.
        confirmacao = input("\n Deseja adicionar este capítulo ao livro? (sim/não): ").lower()
        if confirmacao == "sim":
            # Adiciona o novo capítulo ao rascunho.
            rascunho_atual += "\n\n" + proximo_capitulo
            # Salva o rascunho atualizado.
            salvar_rascunho(rascunho_atual, nome_arquivo_rascunho)
            print("\n✅ Capítulo adicionado ao rascunho.")
        else:
            print("\n❌ Capítulo descartado.")
    # Se o usuário não digitou nada, pede para fornecer instruções.
    else:
        print("\n⚠️ Por favor, forneça instruções para o próximo capítulo.")